In [1]:
# %pip install astropy
# %pip install scipy
# %pip install matplotlib
# %pip install skyfield
# Note: Astroquery no longer needed since we use a local star catalog

In [2]:
import numpy as np
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import math
from datetime import datetime, timezone, timedelta

from instrument_config import TelescopeConfig, SensorConfig
from synthetic_image import (
    get_star_catalog,
    build_synthetic_wcs,
    make_moffat_psf,
    sky_brightness_to_electrons,
    mag_to_flux,
    add_star_trails_fixed_camera
)
from skyfield.api import load, wgs84, Star

In [ ]:
# Telescope and sensor configurations
telescope = TelescopeConfig(
    focal_length_mm=2040.0,
    aperture_mm=204.0,
    image_circle_mm=50.0
)

sensor = SensorConfig(
    pixel_size_um=4.95,
    sensor_width_px=4144,
    sensor_height_px=2822,
    full_well_capacity_e=63700,
    quantum_efficiency=0.75,
    read_noise_e=1.2
)

computed_pixscale_arcsec = sensor.compute_pixel_scale_arcsec(telescope)
fov_x_deg, fov_y_deg = sensor.compute_fov_degrees(telescope)

print(f"Pixel scale from config: {computed_pixscale_arcsec:.3f} arcsec/pixel")
print(f"Aperture area from config: {telescope.aperture_area_cm2:.2f} cm^2")
print(f"F-ratio from config: f/{telescope.f_ratio:.2f}")
print(f"Computed FoV: {fov_x_deg:.2f}° (H) x {fov_y_deg:.2f}° (V)")

In [4]:
# Define the desired sky region and imaging parameters
ra_center = 88.29533   # degrees
dec_center = -5.364887 # degrees
rotation_degrees = -52.0
max_mag = 14.5
seeing_arcsec = 5.5
exposure_time = 1.0     # seconds (will be overridden by timelapse loop)
sky_mag_per_arcsec2 = 18.5   # mag/arcsec^2
mag_zero_point = 15.5        # Zero point for the camera/telescope optical system.

# Build a synthetic WCS for reference
wcs_obj, wcs_header = build_synthetic_wcs(
    image_width=sensor.sensor_width_px,
    image_height=sensor.sensor_height_px,
    ra_center=ra_center,
    dec_center=dec_center,
    pixscale_arcsec=computed_pixscale_arcsec,
    rotation_degrees=rotation_degrees
)

In [ ]:
# Compute diagonal FoV and search radius
import math
diagonal_fov_deg = math.sqrt(fov_x_deg**2 + fov_y_deg**2)
search_radius_deg = diagonal_fov_deg / 2.0
print(f"Using a search radius of ~{search_radius_deg:.2f} deg")

In [6]:
# Create a Moffat PSF
fwhm_pixels = seeing_arcsec / computed_pixscale_arcsec
psf_kernel = make_moffat_psf(
    size=int(fwhm_pixels * 6),
    fwhm=fwhm_pixels,
    moffat_alpha=4
)

In [ ]:
# Compute sky background in e-/pixel for a single exposure
sky_bg = sky_brightness_to_electrons(
    sky_mag_per_arcsec2,
    plate_scale_arcsec_per_pix=computed_pixscale_arcsec,
    exposure_time=exposure_time,
    quantum_efficiency=sensor.quantum_efficiency,
    aperture_area=telescope.aperture_area_cm2,
    mag_zero_point=mag_zero_point
)
print(f"Sky background (for 1s) ~ {sky_bg:.2f} e-/pixel")

In [ ]:
# We'll compute alt0_guess and az0_guess from RA/Dec center
# so that the timelapse code can keep the camera fixed at that alt/az.
start_time_test = datetime(2025, 1, 10, 9, 0, 0, tzinfo=timezone.utc)
ts_test = load.timescale()
t0_test = ts_test.from_datetime(start_time_test)
planets_test = load('de421.bsp')
earth_test = planets_test['earth']
site_test = earth_test + wgs84.latlon(34.05, -118.25, elevation_m=100.0)
center_star = Star(ra_hours=ra_center/15.0, dec_degrees=dec_center)
app_center = site_test.at(t0_test).observe(center_star).apparent()
alt_center, az_center, dist_center = app_center.altaz()
alt0_guess = alt_center.degrees
az0_guess = az_center.degrees
print(f"Computed alt0={alt0_guess:.2f}°, az0={az0_guess:.2f}° at reference time")

## Timelapse Simulation

In [ ]:
from skyfield.api import load, wgs84, Star
from synthetic_image import (
    get_star_catalog,
    add_star_trails_fixed_camera,
    sky_brightness_to_electrons,
    mag_to_flux
)

# Import the new function that uses a real TLE
from satellite_tools import add_satellite_from_tle

############################################
# Timelapse configuration
############################################
timelapse_duration_minutes = 2
exposure_time_seconds = 2.0  # let's do 2s per frame
num_exposures = int((timelapse_duration_minutes * 60) / exposure_time_seconds)
print(f"Generating {num_exposures} exposures, each {exposure_time_seconds} seconds long, over {timelapse_duration_minutes} minutes.")

base_start_time = datetime(2025, 1, 10, 9, 0, 0, tzinfo=timezone.utc)

alt0_cam = alt0_guess
az0_cam  = az0_guess

# We'll also grab the timescale object here
ts = load.timescale()

# Example TLE for the ISS (just as an example). Replace with your real TLE lines.
tle_line1 = "1 25544U 98067A   23014.51999306  .00006404  00000+0  12003-3 0  9993"
tle_line2 = "2 25544  51.6448 292.0392 0006094 186.3085 173.8248 15.48976570438487"

for i in range(num_exposures):
    current_start = base_start_time + timedelta(seconds=i * exposure_time_seconds)
    current_end   = current_start + timedelta(seconds=exposure_time_seconds)
    midpoint_time = current_start + 0.5 * (current_end - current_start)
    t_mid_skyfield = ts.from_datetime(midpoint_time)

    # Convert Alt/Az to RA/Dec for the camera's alt/az at the midpoint
    # (so the star catalog can be centered properly)
    def altaz_to_radec(alt_deg, az_deg, lat_deg, lon_deg, alt_m, time_skyfield):
        planets = load('de421.bsp')
        earth = planets['earth']
        site = earth + wgs84.latlon(lat_deg, lon_deg, elevation_m=alt_m)
        aa_position = site.at(time_skyfield).from_altaz(alt_degrees=alt_deg, az_degrees=az_deg)
        ra_angle, dec_angle, _dist = aa_position.radec()
        return (ra_angle.hours * 15.0, dec_angle.degrees)

    current_ra_center, current_dec_center = altaz_to_radec(
        alt0_cam, az0_cam,
        34.05, -118.25, 100.0,
        t_mid_skyfield
    )

    # Create a fresh blank image for each exposure
    image_timelapse = np.zeros((sensor.sensor_height_px, sensor.sensor_width_px), dtype=float)

    # Recompute sky_bg for the actual exposure_time_seconds
    current_sky_bg = sky_brightness_to_electrons(
        sky_mag_per_arcsec2,
        plate_scale_arcsec_per_pix=computed_pixscale_arcsec,
        exposure_time=exposure_time_seconds,
        quantum_efficiency=sensor.quantum_efficiency,
        aperture_area=telescope.aperture_area_cm2,
        mag_zero_point=mag_zero_point
    )
    image_timelapse += current_sky_bg

    # Query star catalog near the updated RA/Dec center
    star_catalog = get_star_catalog(
        current_ra_center,
        current_dec_center,
        radius=search_radius_deg,
        max_mag=max_mag,
        local_fits_path="celestial_equator_band.fits"
    )
    if star_catalog is not None and len(star_catalog) > 0:
        # Add star trails for each exposure
        add_star_trails_fixed_camera(
            image=image_timelapse,
            star_catalog=star_catalog,
            lat_deg=34.05,
            lon_deg=-118.25,
            alt_m=100.0,
            start_time=current_start,
            end_time=current_end,
            num_sub_steps=20,
            aperture_area=telescope.aperture_area_cm2,
            quantum_efficiency=sensor.quantum_efficiency,
            mag_zero_point=mag_zero_point,
            psf_kernel=psf_kernel,
            extinction=0.5,
            camera_rotation_deg=-90.0,
            arcsec_per_pix=computed_pixscale_arcsec,
            alt0=alt0_cam,
            az0=az0_cam,
            apply_refraction=False
        )

    # Add a satellite using real TLE data
    add_satellite_from_tle(
        image=image_timelapse,
        wcs_obj=wcs_obj,
        tle_line1=tle_line1,
        tle_line2=tle_line2,
        lat_deg=34.05,
        lon_deg=-118.25,
        alt_m=100.0,
        start_time=current_start,
        end_time=current_end,
        psf_kernel=psf_kernel,
        satellite_mag=3.0,
        aperture_area=telescope.aperture_area_cm2,
        quantum_efficiency=sensor.quantum_efficiency,
        mag_zero_point=mag_zero_point,
        num_steps=20
    )

    # Add noise
    shot_noise_image = np.random.poisson(image_timelapse)
    read_noise_image = np.random.normal(loc=0.0, scale=sensor.read_noise_e, size=image_timelapse.shape)
    final_noisy_image = shot_noise_image + read_noise_image

    # Flip the image horizontally to match prior code usage
    final_noisy_image = np.fliplr(final_noisy_image)

    # Create FITS header
    frame_header = fits.Header()
    frame_header["SIMPLE"] = (True, "conforms to FITS standard")
    frame_header["BITPIX"] = -32
    frame_header["NAXIS"] = 2
    frame_header["EXTEND"] = True
    # Merge the WCS header
    for k, v in wcs_header.items():
        frame_header[k] = v

    # Add metadata fields
    frame_header["EXPTIME"] = (exposure_time_seconds, "Exposure time in seconds")
    frame_header["DATE-OBS"] = (current_start.strftime("%Y-%m-%dT%H:%M:%S"), "Start of exposure")
    frame_header["TELESCOP"] = ("Simulator", "Telescope name")
    frame_header["PIXSCALE"] = (computed_pixscale_arcsec, "Plate scale in arcsec/pixel")
    frame_header["RA"] = (current_ra_center, "Right Ascension in degrees (mid-exposure)")
    frame_header["DEC"] = (current_dec_center, "Declination in degrees (mid-exposure)")
    frame_header["RADECSYS"] = ("ICRS", "Right Ascension and Declination system")
    frame_header["FOVX"] = (fov_x_deg, "Horizontal field of view in degrees")
    frame_header["FOVY"] = (fov_y_deg, "Vertical field of view in degrees")

    hdu_frame = fits.PrimaryHDU(data=final_noisy_image, header=frame_header)
    ts_label = current_start.strftime("%Y%m%d_%H%M%S")
    out_name = f"output/timelapse_frame_{i+1:03d}_{ts_label}.fits"
    hdu_frame.writeto(out_name, overwrite=True)
    print(f"Saved timelapse frame {i+1} of {num_exposures} => {out_name}")

print("Timelapse simulation complete.")

In [ ]:
# Debug/Export: Convert generated FITS images into PNGs and compile them into a movie.
# Requirements: matplotlib, ffmpeg installed.

import os, glob
import matplotlib.pyplot as plt

fits_files = sorted(glob.glob("output/timelapse_frame_*.fits"))
if len(fits_files) == 0:
    print("No timelapse_frame_*.fits files found in output/ .")
else:
    print(f"Found {len(fits_files)} FITS timelapse frames.")

    # 1) Export each FITS to PNG
    for i, fits_path in enumerate(fits_files):
        hdul = fits.open(fits_path)
        data = hdul[0].data
        hdul.close()
        fig, ax = plt.subplots(figsize=(6,4))
        ax.imshow(data, origin='lower', cmap='gray',
                  vmin=data.mean() - 1.5*data.std(), vmax=data.mean() + 3*data.std())
        png_name = fits_path.replace(".fits", ".png")
        plt.savefig(png_name, dpi=100, bbox_inches='tight')
        plt.close(fig)
        print(f"Saved PNG => {png_name}")

    # 2) Use ffmpeg to compile them into an MP4 movie
    print("Compiling PNG frames into timelapse_video.mp4...")
    os.system("ffmpeg -y -framerate 30 -pattern_type glob -i 'output/timelapse_frame_*.png' -c:v libx264 -pix_fmt yuv420p timelapse_video.mp4")
    print("Done generating timelapse_video.mp4")